# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [42]:
# import libraries

import pandas as pd
import numpy as np
from nltk import word_tokenize
import sqlite3
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline,FeatureUnion
from sqlalchemy import create_engine
from nltk.stem import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin

In [3]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql_table('disaster_response', engine)
X = df['message']
Y = df.drop(columns= ['id','message', 'original', 'genre'])

In [4]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """
    tokenize, lemmanize, normalization,strip, removal of stop words
    from the text
    """
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stopWords = set(stopwords.words('english'))
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        if clean_tok not in stopWords:
            clean_tokens.append(clean_tok)
    return clean_tokens
    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [58]:
pipeline = Pipeline([ ('vec', CountVectorizer(tokenizer= tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf' , MultiOutputClassifier(RandomForestClassifier(n_jobs=-1))),])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train,X_test, y_train,y_test = train_test_split(X,Y, test_size=0.2)


In [8]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [61]:
#pipeline.get_params()

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred = pipeline.predict(X_test)
Y_pred = pd.DataFrame(y_pred,columns=Y.columns)
for col in Y.columns:
        print(f'Column Name:{col}\n')
        print(classification_report(y_test[col],Y_pred[col]))


Column Name:related

             precision    recall  f1-score   support

          0       0.64      0.41      0.50      1205
          1       0.83      0.93      0.88      3992
          2       0.29      0.15      0.20        47

avg / total       0.79      0.80      0.79      5244

Column Name:request

             precision    recall  f1-score   support

          0       0.90      0.97      0.93      4360
          1       0.78      0.45      0.57       884

avg / total       0.88      0.89      0.87      5244

Column Name:offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5221
          1       0.00      0.00      0.00        23

avg / total       0.99      1.00      0.99      5244

Column Name:aid_related

             precision    recall  f1-score   support

          0       0.74      0.86      0.80      3043
          1       0.75      0.59      0.66      2201

avg / total       0.75      0.75      0.74      5244

C

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
    'tfidf__use_idf': (True,False),
    'clf__estimator__n_estimators': [25,50]  
     }
                          
cv = GridSearchCV(pipeline, param_grid = parameters)

cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = cv.predict(X_test)
Y_pred = pd.DataFrame(y_pred,columns=Y.columns)
for col in Y.columns:
        """
        make print each categorical column accuracy, f1 score, and recall
        """
        print(f'Column Name:{col}\n')
        print(classification_report(y_test[col],Y_pred[col]))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [10]:
class TextLengthExtractor(BaseEstimator,TransformerMixin):
    """
    An estimator that count the text length of each cell in the X
    """
    def fit(self, X, y= None):
        """
        return self
        """
        return self
    def transform(self,X):
        """
        count the length of each cell in the X
        """
        X_length = pd.Series(X).str.len()
        
        return pd.DataFrame(X_length)
           

In [55]:
def build_model():
    """
    Build pipeline with Tfidf, length of text and OneVsRestClassifier and GridSearchCV
    """
    pipeline = Pipeline([
        ('feature', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer = tokenize)),
                ('tfidf', TfidfTransformer())
                
            ])),
            ('text_len', TextLengthExtractor())
        ])),
        ('clf', MultiOutputClassifier(OneVsRestClassifier(MultinomialNB( fit_prior=True, class_prior=None)))),
    ])
    
    # Set up the search grid
    parameters = {'feature__text_pipeline__tfidf__use_idf': (True,False),
                   'clf__estimator__estimator__alpha': (1e-2, 1e-3)
    }
    
    # Initialize GridSearch cross validation object
    cv = GridSearchCV(pipeline, param_grid=parameters)

    return cv  

In [56]:
model = build_model()
print('Training model...')
model.fit(X_train, y_train)

Training model...


/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training example

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('feature', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df...tinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
          n_jobs=1),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'feature__text_pipeline__tfidf__use_idf': (True, False), 'clf__estimator__estimator__alpha': (0.01, 0.001)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [60]:
# Use model to predict
Y_pred = model.predict(X_test)
# Turn prediction into DataFrame
Y_pred = pd.DataFrame(Y_pred,columns=Y.columns)
# For each category column, print performance
for col in Y.columns:
    print(f'Column Name:{col}\n')
    print(classification_report(y_test[col],Y_pred[col]))

Column Name:related

             precision    recall  f1-score   support

          0       0.72      0.39      0.51      1220
          1       0.83      0.95      0.89      3985
          2       0.78      0.36      0.49        39

avg / total       0.81      0.82      0.80      5244

Column Name:request

             precision    recall  f1-score   support

          0       0.91      0.95      0.93      4320
          1       0.71      0.58      0.64       924

avg / total       0.88      0.88      0.88      5244

Column Name:offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5220
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5244

Column Name:aid_related

             precision    recall  f1-score   support

          0       0.79      0.79      0.79      3092
          1       0.70      0.69      0.70      2152

avg / total       0.75      0.75      0.75      5244

C

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [62]:
import pickle
# Save to file in the current working directory
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.